<a href="https://colab.research.google.com/github/Satomi-123/thanapol.p_VueLab/blob/main/GGEZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import multiprocessing as mp
import time
import pandas as pd
from datetime import date

def remove_sign(lst,col,sign,new_sign):
    for i in range(len(lst)):
        check_com = list(lst[i])
        check_com[col] = lst[i][col].replace(sign,new_sign)
        lst[i] = tuple(check_com)

def remove_dash(lst,col):
    for i in range(len(lst)):
        if lst[i][col] == '-' and len(lst[i][col]) <=1:
            check_com = list(lst[i])
            check_com[col] = lst[i][col].replace('-','0')
            lst[i] = tuple(check_com)


def combine_list_sol2(lst1,lst2):
    #add list into element of tuple
    return [a + (b,) for a, b in zip(lst1, lst2)]

def main_transform(lst,stock_name):
    remove_sign(lst,5,'+','')
    remove_sign(lst,6,'+','')
    remove_sign(lst,7,',','')
    remove_sign(lst,8,',','')
    

    for j in range(1,9):
        remove_dash(lst,j)

    final_list = combine_list_sol2(lst,stock_name)
    remove_sign(final_list,9,'%26','&')

    return final_list
    
def remove_not_use_link(link):
    with_link = '<a href="stockquotation.do?symbol='
    filter_link = [x for x in link if x.startswith(with_link)]
    new_set = [x.replace('<a href="stockquotation.do?symbol=', '').replace('&amp;ssoPageId=1&amp;language=en&amp;country=US">','').replace('\r\n    </a>','') for x in filter_link]

    all_stock = []
    for i in new_set:
        t = i.find("\r")
        all_stock.append(i[:t])

    return all_stock

#---------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------

def main_pull_data(stock): 
    pages = range(0,3)
    real_row = []
    c_stock = []
    #get html data
    for page in pages:
        url = "https://www.set.or.th/set/historicaltrading.do?symbol="+stock+"&page="+ str(page)+"&language=en&country=TH&type=trading"
        page1 = requests.get(url)
        soup = BeautifulSoup(page1.text,"lxml") 
        # #get table data
        table = soup.find('table',{'class':'table table-hover table-info'})

        if table == None:
            continue
        else:
            for row in table.find_all('tr')[1:]: 
                data = row.find_all('td')
                row_data = [td.text.strip() for td in data]
                c_stock.append(stock)
                real_row.append(tuple(row_data))

    return main_transform(real_row,c_stock)

def pull_header(stock):
    header = []
    url = "https://www.set.or.th/set/historicaltrading.do?symbol="+stock+"&page=0&language=en&country=TH&type=trading"
    page1 = requests.get(url)
    soup = BeautifulSoup(page1.text,"lxml") 
    # #get table data
    table = soup.find('table',{'class':'table table-hover table-info'})
    for i in table.find_all('th'):
        title = i.text.strip()
        title = title.replace("'","")
        header.append(title)
    return header



def pull_name_stock(type_stock):
    url = "https://marketdata.set.or.th/mkt/sectorquotation.do?language=en&country=TH&market=SET&sector="+str.upper(type_stock)
    page1 = requests.get(url)
    soup = BeautifulSoup(page1.text,"lxml") 
    # #get table data
    html_code = soup.find('div',{'id':'maincontent'})

    link = []
    for a in html_code.find_all('a',href=True):
        link.append(str(a))

    return remove_not_use_link(link)


final_data = []
header = []
user_input = input("Enter Stock menu [ALL / ONE]: ")
if str.upper(user_input) == 'ALL':
    type_input = input("Enter Stock type: ")
    stock_name = pull_name_stock(type_input)
    for i in stock_name:
        final_data.extend(main_pull_data(i))
        if len(header) == 9:
            continue
        else:
            header.extend(pull_header(i))

    header.append("Stock name")
    df_final = pd.DataFrame(final_data,columns=header)
    print(df_final)
    df_final.to_csv('%s_in_6_month_En_update.csv' %(str.upper(type_input)),header=True,index=False,encoding="utf-8")
elif str.upper(user_input) == 'ONE':
    stock_input = input("Enter Stock name: ")
    final_data = main_pull_data(str.upper(stock_input))
    header.extend(pull_header(stock_input))
    header.append("Stock name")
    df_final = pd.DataFrame(final_data,columns=header)
    print(df_final)
    df_final.to_csv('%s_in_6_month_En_update.csv'%(str.upper(stock_input)),header=True,index=False,encoding="utf-8")



Enter Stock menu [ALL / ONE]: all
Enter Stock type: food
            Date   Open  ... Total Value(000 Baht) Stock name
0     27/08/2021   8.80  ...              53141.03      APURE
1     26/08/2021   8.55  ...              40080.45      APURE
2     25/08/2021   9.15  ...             276194.79      APURE
3     24/08/2021   8.65  ...             160411.99      APURE
4     23/08/2021   8.65  ...              38982.50      APURE
...          ...    ...  ...                   ...        ...
5563  05/03/2021  12.60  ...               5368.15        ZEN
5564  04/03/2021  13.10  ...              19830.69        ZEN
5565  03/03/2021  13.20  ...              18895.66        ZEN
5566  02/03/2021  12.80  ...              52471.24        ZEN
5567  01/03/2021  12.20  ...              41756.63        ZEN

[5568 rows x 10 columns]
